<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/llm/gradient_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 梯度基础模型


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-embeddings-langchain
%pip install llama-index-llms-gradient

In [ ]:
!pip install llama-index

In [ ]:
%pip install llama-index --quiet
%pip install gradientai --quiet

In [ ]:
import os

os.environ["GRADIENT_ACCESS_TOKEN"] = "{GRADIENT_ACCESS_TOKEN}"
os.environ["GRADIENT_WORKSPACE_ID"] = "{GRADIENT_WORKSPACE_ID}"

## 流程 1：直接查询Gradient LLM


In [ ]:
from llama_index.llms.gradient import GradientBaseModelLLM

llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

In [ ]:
result = llm.complete("Can you tell me about large language models?")
print(result)

## 流程2：梯度LLM辅助的检索增强生成（RAG）


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.core.node_parser import SentenceSplitter

#### 下载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

### 加载文档


In [ ]:
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

### 配置Gradient LLM

在这个notebook中，我们将学习如何配置Gradient Language Model (LLM)。 Gradient LLM 是一个基于transformer的语言模型，可以用于各种自然语言处理任务。我们将学习如何配置模型的参数，以便在特定任务中进行微调和训练。


In [ ]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())
splitter = SentenceSplitter(chunk_size=1024)

### 设置和查询索引


In [ ]:
index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter], embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)

In [ ]:
response = query_engine.query(
    "What did the author do after his time at Y Combinator?"
)
print(response)